In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Setting the random seed for all the successive randomized operations
SEED = 1234
tf.random.set_seed(SEED)
np.random.seed(SEED)

# Is GPU Working?
tf.test.gpu_device_name()

'/device:GPU:0'

Obtain the Dataset and the GloVe Embedding through a Drive Direct Download link.

In [2]:
!wget --no-check-certificate -r "https://drive.google.com/uc?export=download&id=1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt" -O "BRM-emot-submit-val-arousal.csv"
!wget --no-check-certificate -r "https://drive.google.com/uc?export=download&id=1XlfVSprLu3nj3_hsuAui2d1inFTnhFPX" -O "imagenet1000_clsidx_to_labels.json"
!wget --no-check-certificate -r "https://drive.google.com/uc?export=download&id=1GXqL5zG0PHjX1BDATQCiLRIODKuSpOVn" -O "Glove.zip"

!unzip "/content/Glove.zip"

will be placed in the single file you specified.

--2021-09-08 11:25:03--  https://drive.google.com/uc?export=download&id=1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt
Resolving drive.google.com (drive.google.com)... 142.251.8.139, 142.251.8.102, 142.251.8.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.8.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ete4ri8fu1mrbuger2enrs3mlaka0gtc/1631100300000/16730855647952832235/*/1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt?e=download [following]
--2021-09-08 11:25:05--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ete4ri8fu1mrbuger2enrs3mlaka0gtc/1631100300000/16730855647952832235/*/1yXMBQWxGvTYmndxuREZ9N86l1s43lFRt?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting t

Read the BRM Dataset into a Pandas Dataframe: from all the available columns we retain only the mean Valence and Arousal values.

In [3]:
df = pd.read_csv('BRM-emot-submit-val-arousal.csv')

df = df[['Word', 'V.Mean.Sum', 'A.Mean.Sum']].astype(str)
df.head()

,Word,V.Mean.Sum,A.Mean.Sum
0,aardvark,6.26,2.41
1,abalone,5.3,2.65
2,abandon,2.84,3.73
3,abandonment,2.63,4.95
4,abbey,5.85,2.2


Building the training set extracting from the DataFrame all the words in a list.

In [4]:
text = df['Word'].tolist()

Building the traininig regression values.

In [5]:
y = df[['V.Mean.Sum', 'A.Mean.Sum']]

# Scale the Valence and Arousal values between -1 and 1
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
y = scaler.fit_transform(y)

# Read the the GloVe Embedding from file
GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.

In [6]:
glove_vectors = dict()

file = open('glove.6B.200d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:])
    glove_vectors[word] = vectors
file.close()

In [7]:
len(glove_vectors.keys())

400001

# Apply the Tokenization to GloVe

Build a dictionary using the the words contained in GloVe then apply tokenization

In [8]:
# Build the dictionary
glove_dict = list(glove_vectors.keys())

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Apply the Tokenization to the GloVe dictionary
token = Tokenizer()
token.fit_on_texts(glove_dict)

In [10]:
vocab_size = len(token.word_index) + 1
vocab_size

339252

In [11]:
# Apply the Tokenization for each sequence in the training set
encoded_text = token.texts_to_sequences(text)

In [12]:
# Pad the sequence to a maximum length of 5 adding zeros
max_length = 5
X = tf.keras.preprocessing.sequence.pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [13]:
# Build the embedding matrix picking the embedding values from the GloVe dictionary
word_vector_matrix = np.zeros((vocab_size, 200))

for word, index in token.word_index.items():
    vector = glove_vectors.get(word)
    word_vector_matrix[index] = vector

# Split the Training Set in Training and Validation Data
We used the sklearn `train_test_split` method to split the training data into "training set" and "validation set". The proportion for the split is:
*   90% for training
*   5% for validation
*   5% for testing

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=SEED, test_size=0.05)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=SEED, test_size=0.05)

# Building the Regressor Model
On top of the Embedding Layer (embedding dimension of 200), build the Regression Network:
*  Conv1D(64, 8)
*  MaxPooling1D(2)
*  GlobalAveragingPooling1D
*  Dense(256)
*  Dense(128)

In [16]:
vec_size = 200

lr = 1e-4
dropout= 0.5

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(vocab_size, vec_size, input_length=max_length, weights=[word_vector_matrix], trainable=False))

model.add(tf.keras.layers.Conv1D(64, 8, activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling1D(2))
model.add(tf.keras.layers.Dropout(dropout))

model.add(tf.keras.layers.GlobalAveragePooling1D())

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(dropout))

model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dense(2, activation='linear'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 200)            67850400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 64)             102464    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 2, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 64)             0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)              

Defining the Loss Function and the Optimizer to be used during training.

In [17]:
loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [18]:
# Validation metrics
# ------------------
metrics = [tf.keras.metrics.MeanSquaredError()]

Compiling the Model.

In [19]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [20]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
393/393 [==============================] - 31s 5ms/step - loss: 0.0992 - mean_squared_error: 0.0992 - val_loss: 0.0807 - val_mean_squared_error: 0.0807
Epoch 2/50
393/393 [==============================] - 2s 4ms/step - loss: 0.0800 - mean_squared_error: 0.0800 - val_loss: 0.0716 - val_mean_squared_error: 0.0716
Epoch 3/50
393/393 [==============================] - 2s 4ms/step - loss: 0.0718 - mean_squared_error: 0.0718 - val_loss: 0.0678 - val_mean_squared_error: 0.0678
Epoch 4/50
393/393 [==============================] - 2s 4ms/step - loss: 0.0677 - mean_squared_error: 0.0677 - val_loss: 0.0652 - val_mean_squared_error: 0.0652
Epoch 5/50
393/393 [==============================] - 2s 4ms/step - loss: 0.0646 - mean_squared_error: 0.0646 - val_loss: 0.0643 - val_mean_squared_error: 0.0643
Epoch 6/50
393/393 [==============================] - 2s 5ms/step - loss: 0.0623 - mean_squared_error: 0.0623 - val_loss: 0.0632 - val_mean_squared_error: 0.0632
Epoch 7/50
393/393 [=======

# Inference on the Test Set
We perform an inference on the test set to evaluate the regression performance.

In [21]:
# Before predictions, data needs to be properly encoded
def get_encode(x):
    x = token.texts_to_sequences(x)
    x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=max_length, padding='post')
    return x

In [22]:
# Generate predictions
preds = model.predict(X_test)
# Generate baseline predictions
preds_baseline = np.zeros(y_test.shape) + np.mean(y,axis=0) 

In [23]:
# Mean Absolute Error for predictions
mae = np.mean(np.abs(preds - y_test),axis=0)
# Mean Squared Error for predictions
mse = np.mean(np.square(preds - y_test),axis=0)

# Mean Absolute Error for baseline 
mae_baseline = np.mean(np.abs(preds_baseline - y_test),axis=0)
# Mean Squared Error for baseline 
mse_baseline = np.mean(np.square(preds_baseline - y_test),axis=0)

print('Valence Mean Absolute Error: ' , mae[0], ' VS ', 'Baseline: ', mae_baseline[0], '\n')
print('Arousal Mean Absolute Error: ' , mae[1], ' VS ', 'Baseline: ', mae_baseline[1], '\n')

print('Valence Mean Squared Error: ' , mse[0], ' VS ', 'Baseline: ', mse_baseline[0], '\n')
print('Arousal Mean Squared Error: ' , mse[1], ' VS ', 'Baseline: ', mse_baseline[1], '\n')

Valence Mean Absolute Error:  0.1719224657281219  VS  Baseline:  0.28411726162564677 

Arousal Mean Absolute Error:  0.18310803658288619  VS  Baseline:  0.22336753658178057 

Valence Mean Squared Error:  0.049909606709977385  VS  Baseline:  0.12174314478951893 

Arousal Mean Squared Error:  0.0526859644835051  VS  Baseline:  0.07798782065098707 



In [27]:
print(model.name , 'Coefficient of determination R2:' , 1 - mse/mse_baseline )

sequential_1 Coefficient of determination R2: [0.59004175 0.32443343]


 # Inference on ImageNet
 The following code infers the Valence and Arousal values for the class labels contained in *ImageNet*

Open the ImageNet list of class labels and extract their values in a list

In [24]:
import json

with open('imagenet1000_clsidx_to_labels.json') as json_file:
    imagenet_classes = json.load(json_file)

imagenet_filtered_classes = dict()

for key, value in imagenet_classes.items():
    str_list = value.split(',')
    imagenet_filtered_classes[key] = str_list[0]

Apply the Regression model to the ImageNet Classes

In [25]:
imagenet_va = dict()

for key, value in imagenet_filtered_classes.items():
    imagenet_va[key] = model.predict(get_encode([value]))[0]

In [28]:
imagenet_va   

{'0': array([-0.03559431, -0.2518264 ], dtype=float32),
 '1': array([ 0.2961953 , -0.37648585], dtype=float32),
 '2': array([ 0.3457181 , -0.40715924], dtype=float32),
 '3': array([-0.03422883,  0.21538484], dtype=float32),
 '4': array([-0.03512327, -0.21857832], dtype=float32),
 '5': array([ 0.22215647, -0.24976894], dtype=float32),
 '6': array([ 0.08968339, -0.1434048 ], dtype=float32),
 '7': array([-0.22038074,  0.0957353 ], dtype=float32),
 '8': array([ 0.1506823 , -0.41966987], dtype=float32),
 '9': array([ 0.13252029, -0.29790065], dtype=float32),
 '10': array([-0.04725845,  0.06197202], dtype=float32),
 '11': array([ 0.04961646, -0.2751966 ], dtype=float32),
 '12': array([ 0.1622861 , -0.41484565], dtype=float32),
 '13': array([ 0.12329349, -0.2496436 ], dtype=float32),
 '14': array([ 0.21242435, -0.23076554], dtype=float32),
 '15': array([ 0.05957909, -0.34458798], dtype=float32),
 '16': array([-0.02881012, -0.24882902], dtype=float32),
 '17': array([ 0.04216683, -0.19846986], 